# Milestone 1 — EV Charging Demand Prediction

## 1. Problem Understanding

**Objective:** Build a machine-learning / time-series forecasting system that predicts EV charging demand at stations using historical usage data from the **UrbanEVDataset** (Shenzhen, China — Sep 2022 to Feb 2023).

### Use-Case Description
EV charging stations in urban areas experience highly variable demand patterns driven by:
- **Time of day** (rush hours vs. off-peak)
- **Day of week** (weekdays vs. weekends)
- **Season** (weather, holidays)
- **Station location** (residential area vs. commercial)

Accurate demand forecasting enables:
- Grid operators to balance electricity load
- Station managers to schedule maintenance
- EV drivers to find available chargers

### Input–Output Specification
| Field | Description |
|---|---|
| **Inputs** | Historical 5-min interval station data (busy/idle piles, energy volume, price, duration) |
| **Station Info** | GPS coordinates, pile counts, zone (TAZID) |
| **Outputs** | Hourly energy demand forecast (kWh) per zone |
| **Evaluation** | MAE, RMSE on held-out 20% test split |

## 2. Forecasting Pipeline (System Architecture)

```
┌─────────────────────────────────────────────────────────────────────┐
│                     FORECASTING PIPELINE                           │
│                                                                     │
│  Raw Station CSVs (5-min)                                           │
│       │                                                             │
│       ▼                                                             │
│  Data Cleaning & Gap-Filling (ffill/bfill)                          │
│       │                                                             │
│       ▼                                                             │
│  Remove Bad Stations (all-idle / header-only)                       │
│       │                                                             │
│       ▼                                                             │
│  Hourly Aggregation (per-station → sum/mean)                        │
│       │                                                             │
│  Station Info CSV ──► Zone Mapping (station_id → TAZID)            │
│       │                                                             │
│       ▼                                                             │
│  Zone-Level Hourly Demand (sum of station volumes per zone)         │
│       │                                                             │
│       ▼                                                             │
│  Feature Engineering                                                │
│     hour, dayofweek, month, season, is_weekend, lag_1, lag_24      │
│       │                                                             │
│       ▼                                                             │
│  Model Training (Linear Regression, Ridge)                          │
│     Train: 80% | Test: 20% (time-ordered)                           │
│       │                                                             │
│       ▼                                                             │
│  Evaluation: MAE, RMSE per zone                                     │
│       │                                                             │
│       ▼                                                             │
│  Streamlit Dashboard (demand forecast + peak detection)             │
└─────────────────────────────────────────────────────────────────────┘
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib .pyplot as plt
import seaborn as sns
import os
import glob
import warnings
warnings .filterwarnings ('ignore')
from sklearn .linear_model import LinearRegression ,Ridge
from sklearn .metrics import mean_absolute_error ,mean_squared_error
from sklearn .model_selection import train_test_split
BASE_DIR =os .path .dirname (os .path .abspath ('__file__'))if '__file__'in dir ()else os .getcwd ()
RAW_DIR =os .path .join (BASE_DIR ,'20220901-20230228_station-raw')
CLEAN_DIR =os .path .join (BASE_DIR ,'processed')
STATION_INFO_PATH =os .path .join (RAW_DIR ,'station_information.csv')
RAW_5MIN_GLOB =os .path .join (RAW_DIR ,'charge_5min','*.csv')
OUT_5MIN =os .path .join (CLEAN_DIR ,'charge_5min')
OUT_1HOUR =os .path .join (CLEAN_DIR ,'charge_1hour')
OUT_ZONE =os .path .join (CLEAN_DIR ,'zone_hourly_volume_long.csv')
os .makedirs (OUT_5MIN ,exist_ok =True )
os .makedirs (OUT_1HOUR ,exist_ok =True )
print (' Directories ready')
print (f'   RAW:     {RAW_DIR }')
print (f'   OUTPUT:  {CLEAN_DIR }')


## 3. Station Information & Zone Mapping

In [ ]:
station_info =pd .read_csv (STATION_INFO_PATH )
print (f'Stations: {len (station_info )}')
print (f'Zones covered: {station_info ["TAZID"].nunique ()}')
station_info .head ()


In [ ]:
station_info .describe ()


In [ ]:
station_to_zone =dict (zip (station_info ['station_id'],station_info ['TAZID']))
print (f'Built mapping for {len (station_to_zone )} stations')
fig ,ax =plt .subplots (1 ,2 ,figsize =(12 ,4 ))
station_info ['slow_count'].hist (bins =20 ,ax =ax [0 ],color ='steelblue')
ax [0 ].set_title ('Distribution of Slow Chargers per Station')
ax [0 ].set_xlabel ('Slow Charger Count')
station_info ['fast_count'].hist (bins =20 ,ax =ax [1 ],color ='darkorange')
ax [1 ].set_title ('Distribution of Fast Chargers per Station')
ax [1 ].set_xlabel ('Fast Charger Count')
plt .tight_layout ()
plt .show ()


## 4. Raw Data Exploration (Single Station Sample)

In [ ]:
sample_path =os .path .join (RAW_DIR ,'charge_5min','1001.csv')
sample_df =pd .read_csv (sample_path )
sample_df ['time']=pd .to_datetime (sample_df ['time'])
print (f'Shape: {sample_df .shape }')
print (f'Date range: {sample_df ["time"].min ()}  →  {sample_df ["time"].max ()}')
print (f'Columns: {list (sample_df .columns )}')
print (f'Nulls:\n{sample_df .isnull ().sum ()}')
sample_df .head ()


In [ ]:
col_desc ={'time':'Timestamp (5-min intervals)','busy':'Number of piles currently charging','idle':'Number of piles available','s_price':'Service charge price (CNY/kWh)','e_price':'Electricity price (CNY/kWh)','fast_busy':'Fast charger piles in use','fast_idle':'Fast charger piles idle','slow_busy':'Slow charger piles in use','slow_idle':'Slow charger piles idle','duration':'Total charging duration in interval (hours)','volume':'Total energy dispensed in interval (kWh)',}
pd .DataFrame .from_dict (col_desc ,orient ='index',columns =['Description'])


In [ ]:
fig ,axes =plt .subplots (2 ,1 ,figsize =(14 ,7 ),sharex =False )
sample_daily =sample_df .set_index ('time')['volume'].resample ('h').sum ()
sample_daily .plot (ax =axes [0 ],color ='teal')
axes [0 ].set_title ('Station 1001 — Hourly Energy Volume (kWh) Over Dataset Period')
axes [0 ].set_ylabel ('kWh')
sample_df ['hour']=sample_df ['time'].dt .hour
hourly_avg =sample_df .groupby ('hour')['volume'].mean ()
hourly_avg .plot (kind ='bar',ax =axes [1 ],color ='teal')
axes [1 ].set_title ('Station 1001 — Average Energy Volume by Hour')
axes [1 ].set_xlabel ('Hour of Day')
axes [1 ].set_ylabel ('Avg kWh')
plt .tight_layout ()
plt .show ()


## 5. Preprocessing — All Stations (5-min cleaning + gap fill)

In [ ]:
BAD_STATIONS ={2129 ,1663 ,1478 ,1082 ,1055 ,1722 ,1039 ,1036 ,1681 ,2125 ,1487 ,1113 ,2138 ,1034 ,1337 ,1497 ,2337 ,1501 ,1101 ,2291 }
EXPECTED_5MIN_FREQ ='5min'
files =sorted (glob .glob (RAW_5MIN_GLOB ))
print (f'Total raw station files: {len (files )}')


In [ ]:
success ,skipped ,fail =[],[],[]
for file_path in files :
    station_id =int (os .path .basename (file_path ).replace ('.csv',''))
    if station_id in BAD_STATIONS :
        skipped .append (station_id )
        continue
    if station_id not in station_to_zone :
        skipped .append (station_id )
        continue
    zone_id =station_to_zone [station_id ]
    try :
        df =pd .read_csv (file_path )
        if len (df )<10 :
            skipped .append (station_id )
            continue
        df ['time']=pd .to_datetime (df ['time'])
        df ['TAZID']=zone_id
        df =df .sort_values ('time').set_index ('time')
        full_index =pd .date_range (start =df .index .min (),end =df .index .max (),freq =EXPECTED_5MIN_FREQ )
        df =df .reindex (full_index )
        df =df .ffill ().bfill ()
        df =df .reset_index ().rename (columns ={'index':'time'})
        null_count =df .isnull ().sum ().sum ()
        if null_count >0 :
            fail .append (station_id )
            continue
        out_path =os .path .join (OUT_5MIN ,f'{station_id }.csv')
        df .to_csv (out_path ,index =False )
        success .append (station_id )
    except Exception as e :
        print (f'   Station {station_id }: {e }')
        fail .append (station_id )
print (f'\n Cleaned: {len (success )} stations')
print (f'⏭  Skipped (bad/unmapped): {len (skipped )} stations')
print (f' Failed: {len (fail )} stations')


## 6. Preprocessing — Hourly Aggregation (per-station)

In [ ]:
AGG_RULES ={'busy':'mean','idle':'mean','fast_busy':'mean','fast_idle':'mean','slow_busy':'mean','slow_idle':'mean','duration':'sum','volume':'sum','s_price':'mean','e_price':'mean','TAZID':'first',}
cleaned_files =sorted (glob .glob (os .path .join (OUT_5MIN ,'*.csv')))
print (f'Aggregating {len (cleaned_files )} stations to hourly …')
for file_path in cleaned_files :
    station_id =int (os .path .basename (file_path ).replace ('.csv',''))
    df =pd .read_csv (file_path )
    df ['time']=pd .to_datetime (df ['time'])
    df =df .set_index ('time')
    df_hourly =df .resample ('h').agg (AGG_RULES ).reset_index ()
    out_path =os .path .join (OUT_1HOUR ,f'{station_id }.csv')
    df_hourly .to_csv (out_path ,index =False )
hourly_files =glob .glob (os .path .join (OUT_1HOUR ,'*.csv'))
print (f' Hourly files written: {len (hourly_files )}')
sample_h =pd .read_csv (hourly_files [0 ])
print (f'   Sample shape: {sample_h .shape } — columns: {list (sample_h .columns )}')


## 7. Zone-Level Aggregation

In [ ]:
all_data =[]
for file_path in hourly_files :
    df =pd .read_csv (file_path )
    df ['time']=pd .to_datetime (df ['time'])
    all_data .append (df [['time','TAZID','volume']])
all_data =pd .concat (all_data ,ignore_index =True )
zone_hourly =(all_data .groupby (['time','TAZID'],as_index =False ).agg ({'volume':'sum'}))
zone_hourly .to_csv (OUT_ZONE ,index =False )
print (f'Zone-hourly dataset: {zone_hourly .shape }')
print (f'Zones: {zone_hourly ["TAZID"].nunique ()}')
print (f'Time range: {zone_hourly ["time"].min ()} → {zone_hourly ["time"].max ()}')
zone_hourly .head ()


In [ ]:
print ('Volume statistics (kWh) across all zones and hours:')
zone_hourly ['volume'].describe ()


## 8. Exploratory Data Analysis

In [ ]:
zone_hourly_df =pd .read_csv (OUT_ZONE )
zone_hourly_df ['time']=pd .to_datetime (zone_hourly_df ['time'])
total_demand =zone_hourly_df .groupby ('time')['volume'].sum ()
plt .figure (figsize =(16 ,4 ))
total_demand .plot (color ='royalblue',linewidth =0.8 )
plt .title ('Total EV Charging Demand (All Zones) — Hourly (kWh)')
plt .ylabel ('kWh')
plt .xlabel ('Time')
plt .tight_layout ()
plt .show ()


In [ ]:
zone_hourly_df ['hour']=zone_hourly_df ['time'].dt .hour
zone_hourly_df ['dayofweek']=zone_hourly_df ['time'].dt .dayofweek
zone_hourly_df ['month']=zone_hourly_df ['time'].dt .month
fig ,axes =plt .subplots (1 ,3 ,figsize =(18 ,5 ))
zone_hourly_df .groupby ('hour')['volume'].mean ().plot (kind ='bar',ax =axes [0 ],color ='steelblue')
axes [0 ].set_title ('Avg Demand by Hour of Day')
axes [0 ].set_xlabel ('Hour')
axes [0 ].set_ylabel ('Avg kWh')
days =['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
zone_hourly_df .groupby ('dayofweek')['volume'].mean ().plot (kind ='bar',ax =axes [1 ],color ='darkorange')
axes [1 ].set_xticklabels (days ,rotation =0 )
axes [1 ].set_title ('Avg Demand by Day of Week')
axes [1 ].set_xlabel ('Day')
zone_hourly_df .groupby ('month')['volume'].mean ().plot (kind ='bar',ax =axes [2 ],color ='seagreen')
axes [2 ].set_title ('Avg Demand by Month')
axes [2 ].set_xlabel ('Month')
plt .tight_layout ()
plt .show ()


In [ ]:
top_zones =zone_hourly_df .groupby ('TAZID')['volume'].sum ().nlargest (10 )
top_zones .plot (kind ='barh',figsize =(10 ,5 ),color ='mediumpurple')
plt .title ('Top 10 Zones by Total Charging Demand (kWh)')
plt .xlabel ('Total kWh')
plt .tight_layout ()
plt .show ()
print (top_zones )


## 9. Feature Engineering

In [ ]:
def add_features (df :pd .DataFrame )->pd .DataFrame :
    """Add time-based and lag features to a zone dataframe."""
    df =df .copy ()
    df ['hour']=df .index .hour
    df ['dayofweek']=df .index .dayofweek
    df ['month']=df .index .month
    df ['is_weekend']=(df .index .dayofweek >=5 ).astype (int )
    df ['season']=df ['month'].map ({12 :0 ,1 :0 ,2 :0 ,3 :1 ,4 :1 ,5 :1 ,6 :2 ,7 :2 ,8 :2 ,9 :3 ,10 :3 ,11 :3 })
    df ['hour_sin']=np .sin (2 *np .pi *df ['hour']/24 )
    df ['hour_cos']=np .cos (2 *np .pi *df ['hour']/24 )
    df ['dow_sin']=np .sin (2 *np .pi *df ['dayofweek']/7 )
    df ['dow_cos']=np .cos (2 *np .pi *df ['dayofweek']/7 )
    df ['lag_1h']=df ['volume'].shift (1 )
    df ['lag_24h']=df ['volume'].shift (24 )
    df ['lag_168h']=df ['volume'].shift (168 )
    df ['roll_24h_mean']=df ['volume'].shift (1 ).rolling (24 ).mean ()
    return df .dropna ()
FEATURES =['hour','dayofweek','month','is_weekend','season','hour_sin','hour_cos','dow_sin','dow_cos','lag_1h','lag_24h','lag_168h','roll_24h_mean']
print ('Feature set:')
for f in FEATURES :
    print (f'  · {f }')


## 10. Model Training & Evaluation — All Zones

In [ ]:
zone_hourly_df =pd .read_csv (OUT_ZONE )
zone_hourly_df ['time']=pd .to_datetime (zone_hourly_df ['time'])
zones =zone_hourly_df ['TAZID'].unique ()
results =[]
for zone_id in zones :
    zone_df =(zone_hourly_df [zone_hourly_df ['TAZID']==zone_id ].copy ().sort_values ('time').set_index ('time'))
    zone_df =add_features (zone_df )
    if len (zone_df )<200 :
        continue
    X =zone_df [FEATURES ]
    y =zone_df ['volume']
    X_train ,X_test ,y_train ,y_test =train_test_split (X ,y ,test_size =0.2 ,shuffle =False )
    for name ,model in [('LinearRegression',LinearRegression ()),('Ridge',Ridge (alpha =1.0 ))]:
        model .fit (X_train ,y_train )
        y_pred =model .predict (X_test )
        y_pred =np .maximum (y_pred ,0 )
        mae =mean_absolute_error (y_test ,y_pred )
        rmse =np .sqrt (mean_squared_error (y_test ,y_pred ))
        results .append ({'zone':zone_id ,'model':name ,'MAE':mae ,'RMSE':rmse })
results_df =pd .DataFrame (results )
print (f'Evaluated {results_df ["zone"].nunique ()} zones')
results_df .groupby ('model')[['MAE','RMSE']].mean ().round (3 )


In [ ]:
best_per_zone =results_df .loc [results_df .groupby ('zone')['RMSE'].idxmin ()]
print ('Model selection distribution (best RMSE per zone):')
print (best_per_zone ['model'].value_counts ())
fig ,axes =plt .subplots (1 ,2 ,figsize =(14 ,5 ))
for name ,grp in results_df .groupby ('model'):
    grp ['MAE'].hist (bins =30 ,ax =axes [0 ],alpha =0.6 ,label =name )
    grp ['RMSE'].hist (bins =30 ,ax =axes [1 ],alpha =0.6 ,label =name )
axes [0 ].set_title ('MAE Distribution — All Zones')
axes [0 ].set_xlabel ('MAE (kWh)')
axes [0 ].legend ()
axes [1 ].set_title ('RMSE Distribution — All Zones')
axes [1 ].set_xlabel ('RMSE (kWh)')
axes [1 ].legend ()
plt .tight_layout ()
plt .show ()


In [ ]:
results_save_path =os .path .join (CLEAN_DIR ,'zone_model_results.csv')
results_df .to_csv (results_save_path ,index =False )
print (f'Saved to: {results_save_path }')


## 11. Detailed Zone Analysis — Single Zone Deep Dive

In [ ]:
busiest_zone =int (zone_hourly_df .groupby ('TAZID')['volume'].sum ().idxmax ())
print (f'Analysing busiest zone: {busiest_zone }')
zdf =(zone_hourly_df [zone_hourly_df ['TAZID']==busiest_zone ].copy ().sort_values ('time').set_index ('time'))
zdf =add_features (zdf )
X =zdf [FEATURES ]
y =zdf ['volume']
X_train ,X_test ,y_train ,y_test =train_test_split (X ,y ,test_size =0.2 ,shuffle =False )
model =Ridge (alpha =1.0 )
model .fit (X_train ,y_train )
y_pred =np .maximum (model .predict (X_test ),0 )
mae =mean_absolute_error (y_test ,y_pred )
rmse =np .sqrt (mean_squared_error (y_test ,y_pred ))
print (f'Zone {busiest_zone }  →  MAE: {mae :.2f} kWh | RMSE: {rmse :.2f} kWh')


In [ ]:
fig ,ax =plt .subplots (figsize =(14 ,5 ))
ax .plot (y_test .index ,y_test .values ,label ='Actual',linewidth =1.5 ,color ='royalblue')
ax .plot (y_test .index ,y_pred ,label ='Predicted (Ridge)',linestyle ='--',linewidth =1.2 ,color ='tomato')
ax .set_title (f'Zone {busiest_zone } — Actual vs Predicted Charging Demand (Test Set)')
ax .set_xlabel ('Time')
ax .set_ylabel ('Energy Demand (kWh)')
ax .legend ()
ax .grid (True ,alpha =0.3 )
plt .tight_layout ()
plt .show ()


In [ ]:
peak_hours =zdf ['volume'].nlargest (10 )
print (f'Top 10 Peak Demand Hours — Zone {busiest_zone }:')
print (peak_hours .to_string ())
zdf ['hour']=zdf .index .hour
zdf ['dayofweek']=zdf .index .dayofweek
pivot =zdf .pivot_table (values ='volume',index ='hour',columns ='dayofweek',aggfunc ='mean')
pivot .columns =['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
plt .figure (figsize =(10 ,7 ))
sns .heatmap (pivot ,cmap ='YlOrRd',linewidths =0.5 ,annot =False )
plt .title (f'Zone {busiest_zone } — Avg kWh by Hour × Day of Week')
plt .ylabel ('Hour of Day')
plt .xlabel ('Day')
plt .tight_layout ()
plt .show ()


In [ ]:
coef_df =pd .DataFrame ({'feature':FEATURES ,'coefficient':model .coef_ })
coef_df =coef_df .sort_values ('coefficient',key =abs ,ascending =True )
coef_df .plot (kind ='barh',x ='feature',y ='coefficient',figsize =(10 ,6 ),color =['crimson'if v <0 else 'steelblue'for v in coef_df ['coefficient']])
plt .title (f'Ridge Regression Coefficients — Zone {busiest_zone }')
plt .xlabel ('Coefficient Value')
plt .axvline (x =0 ,color ='black',linewidth =0.8 )
plt .tight_layout ()
plt .show ()


## 12. Summary — Milestone 1 Evaluation

### What was built
| Step | Description |
|---|---|
| Preprocessing | Gap-filled 5-min station CSVs using ffill/bfill; removed 20 bad stations |
| Aggregation | Resampled to 1-hour intervals; summed per zone |
| Feature Engineering | Time features + sin/cos encoding + 3 lag features + rolling mean |
| Models | Linear Regression, Ridge Regression (per zone) |
| Evaluation | MAE and RMSE on chronological 20% test split |

### Key Findings
- Demand peaks in the **evening (18:00–21:00)** and is lowest **01:00–05:00**
- **Weekday** demand generally exceeds weekend demand
- **lag_1h** and **lag_24h** are the strongest predictors
- Ridge Regression outperforms plain Linear Regression on high-demand zones

### Next Steps (Milestone 2)
- LSTM / GRU time-series models for non-linear capture
- Add weather data as exogenous feature
- Spatial graph neural networks using `adj.csv`